In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.0'
spark_version = 'spark-3.2.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:13 http://sec

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-01-04 20:16:39--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.1s    

2022-01-04 20:16:40 (5.90 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Video_DVD_v1_00").getOrCreate()

In [4]:
from pyspark import SparkFiles
# Load in data from S3 into a DataFrame
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_DVD_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

In [5]:
video_dvd_df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Video_DVD_v1_00.tsv.gz"), inferSchema=True, sep='\t')
video_dvd_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   27288431| R33UPQQUZQEM8|B005T4ND06|     400024643|Yoga for Movement...|       Video DVD|          5|            3|          3|   N|                Y|This was a gift f...|This was a gift f...| 2015-08-31|
|         US|   13722556|R3IKTNQQPD9662|B004EPZ070|     685335564|  Something Borrowed|       Video DVD|          5|    

In [6]:
#Count number of records in each row

video_dvd_df.count()

5069140

In [8]:
#Drop empty
video_dvd_df = video_dvd_df.dropna()
video_dvd_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   27288431| R33UPQQUZQEM8|B005T4ND06|     400024643|Yoga for Movement...|       Video DVD|          5|            3|          3|   N|                Y|This was a gift f...|This was a gift f...| 2015-08-31|
|         US|   13722556|R3IKTNQQPD9662|B004EPZ070|     685335564|  Something Borrowed|       Video DVD|          5|    

In [9]:
#Customer table 
customer_df = video_dvd_df.select(["customer_id"])
customer_df.show()

+-----------+
|customer_id|
+-----------+
|   27288431|
|   13722556|
|   20381037|
|   24852644|
|   15556113|
|    6132474|
|   48049524|
|    3282516|
|   51771179|
|   31816501|
|   16164990|
|   33386989|
|   45486371|
|   14006420|
|   23411619|
|    3202332|
|   32012808|
|   36507765|
|   13696097|
|    2190805|
+-----------+
only showing top 20 rows



In [10]:
customer_df = customer_df.groupBy("customer_id").count()
customer_df.orderBy("customer_id").select(["customer_id", "count"])
customer_df.show()

+-----------+-----+
|customer_id|count|
+-----------+-----+
|   24423656|    8|
|   24297214|    1|
|   12879980|    3|
|     515450|    2|
|   13313689|    1|
|   15523729|    4|
|    1673863|    1|
|   14552054|    2|
|   45392827|   14|
|   44178035|    1|
|   10522786|   10|
|   44848424|    3|
|   14230926|    1|
|   49243158|  274|
|   49084939|   10|
|   28777148|   41|
|   41836864|    1|
|    5219946|    1|
|   37795150|    1|
|   52081222|    1|
+-----------+-----+
only showing top 20 rows



In [11]:
customer_df = customer_df.withColumnRenamed("count", "customer_count")
customer_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   24423656|             8|
|   24297214|             1|
|   12879980|             3|
|     515450|             2|
|   13313689|             1|
|   15523729|             4|
|    1673863|             1|
|   14552054|             2|
|   45392827|            14|
|   44178035|             1|
|   10522786|            10|
|   44848424|             3|
|   14230926|             1|
|   49243158|           274|
|   49084939|            10|
|   28777148|            41|
|   41836864|             1|
|    5219946|             1|
|   37795150|             1|
|   52081222|             1|
+-----------+--------------+
only showing top 20 rows



In [12]:
customer_df.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: long (nullable = false)



In [13]:
#Products table
products_df = video_dvd_df.select(["product_id", "product_title"])
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B005T4ND06|Yoga for Movement...|
|B004EPZ070|  Something Borrowed|
|B005S9EKCW|Les Miserables (2...|
|B00FC1ZCB4|Alien Anthology a...|
|B002ZG98Z0|  Sex and the City 2|
|B00X8RONBO|When Calls The He...|
|B000CEXFZG|Teen Titans - The...|
|B00ID8H8EW|      Generation War|
|B000TGJ8IU|Troy  (Director's...|
|B00DPMPTDS|Faith Aka the Gre...|
|B00X797LUS|   Revenge: Season 4|
|B00C6MXB42|YOUNG INDIANA JON...|
|B000EZ9084|     Survival Island|
|B000CEXFZG|Teen Titans - The...|
|B00KXEM6XM|     Fargo: Season 1|
|B0036K9CM8|Disney Parks: The...|
|B00G3HOJZ6|Young Detective D...|
|B00V5E7YR2|The Divergent Ser...|
|B005BUA1JY|Barney Miller: Th...|
|B00YCY46VO|Inside Out (Blu-r...|
+----------+--------------------+
only showing top 20 rows



In [14]:
products_df = products_df.dropDuplicates(["product_id"])

In [15]:
products_df.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_title: string (nullable = true)



In [16]:
#Review ID table
review_id_df = video_dvd_df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| R33UPQQUZQEM8|   27288431|B005T4ND06|     400024643| 2015-08-31|
|R3IKTNQQPD9662|   13722556|B004EPZ070|     685335564| 2015-08-31|
|R3U27V5QMCP27T|   20381037|B005S9EKCW|     922008804| 2015-08-31|
|R2TOH2QKNK4IOC|   24852644|B00FC1ZCB4|     326560548| 2015-08-31|
|R2XQG5NJ59UFMY|   15556113|B002ZG98Z0|     637495038| 2015-08-31|
|R1N1KHBRR4ZTX3|    6132474|B00X8RONBO|     896602391| 2015-08-31|
|R3OM9S0TCBP38K|   48049524|B000CEXFZG|     115883890| 2015-08-31|
|R1W4S949ZRCTBW|    3282516|B00ID8H8EW|     977932459| 2015-08-31|
|R18JL1NNQAZFV2|   51771179|B000TGJ8IU|     840084782| 2015-08-31|
|R1LP6PR06OPYUX|   31816501|B00DPMPTDS|     262144920| 2015-08-31|
| RZKBT035JA0UQ|   16164990|B00X797LUS|     883589001| 2015-08-31|
|R253N5W74SM7N3|   33386989|B00C6MXB42|     734735137| 2015-08

In [17]:
review_id_df = review_id_df.dropDuplicates(["review_id"])

In [ ]:
review_id_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: string (nullable = true)



In [18]:
#Vine table
vine_df = video_dvd_df.select(["review_id", "customer_id", "star_rating", "helpful_votes", "total_votes"])
vine_df.show()

+--------------+-----------+-----------+-------------+-----------+
|     review_id|customer_id|star_rating|helpful_votes|total_votes|
+--------------+-----------+-----------+-------------+-----------+
| R33UPQQUZQEM8|   27288431|          5|            3|          3|
|R3IKTNQQPD9662|   13722556|          5|            0|          0|
|R3U27V5QMCP27T|   20381037|          5|            1|          1|
|R2TOH2QKNK4IOC|   24852644|          5|            0|          1|
|R2XQG5NJ59UFMY|   15556113|          5|            0|          0|
|R1N1KHBRR4ZTX3|    6132474|          5|            0|          0|
|R3OM9S0TCBP38K|   48049524|          5|            0|          0|
|R1W4S949ZRCTBW|    3282516|          5|            0|          0|
|R18JL1NNQAZFV2|   51771179|          5|            0|          0|
|R1LP6PR06OPYUX|   31816501|          4|            0|          0|
| RZKBT035JA0UQ|   16164990|          5|            1|          2|
|R253N5W74SM7N3|   33386989|          4|            1|        

In [19]:
vine_df = vine_df.dropDuplicates(["review_id"])

In [20]:
vine_df.count()

5068421

In [21]:
vine_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)

